## load data

In [2]:
import pandas as pd


In [3]:
raw_data = pd.read_csv('./sorted_data.csv')
raw_Free_text = raw_data['Free_text']
raw_diagnosis = raw_data['diagnosis']
raw_stage = raw_data['stage']
raw_prior_therapy = raw_data['prior_therapy']
raw_genetic_mutation = raw_data['genetic_mutation']
raw_additional_remark = raw_data['additional_remark']

In [4]:
raw_Free_text[0], len(raw_Free_text[0])

('No known allergies 1. Lung adenocarcinoma with pleural and BRAIN mets (TPS 60%, MET polysomy, ALK IHC high but FISH negative, EGFR exon 20 insertion (NM_005228.3) p.A763_Y764insFQEA)) to pleural, hilar and mediastinal nodes - Afatinib from 7/11/20 (dose reduced to 30mg on 27/11/20) - CTNO155X2101 May 2022 > EOT 9/9/22 - Bayer21607 22/9/22 > EOT 9/1/23 - Alimta carboplatin from 20/1/23 to 28/4/23 - WBRT June 2023 - Gemcitabine carboplatin from 6/7/23 to 11/1/24 2. Ovarian cyst',
 480)

In [5]:
def is_num(a):
    if a>='0' and a<='9':
        return True
    else: return False

In [6]:
def is_alpbet(chactor):
    if (chactor>='a' and chactor<='z') or (chactor>='A' and chactor<='Z'):
        return True
    else: return False

## split based on the space and the punctuation marks, but remain the '/' to present the date (further update needed for filtering '%' or '.' or more marks)
### -- how about just keep the punctuation marks that following the numbers, both saves the '/' in date and '%' in percent, but it also save the '.' after the number that presenting different notes. (not applied)

In [7]:
def split(Sentence):
    start_index, end_index = 0, 1
    max_length = len(Sentence)
    # print(max_length)
    result = []
    while (end_index < max_length and start_index < max_length):
        if is_num(Sentence[start_index]) or is_alpbet(Sentence[start_index]):
            if is_num(Sentence[end_index]) or is_alpbet(Sentence[end_index]):
                end_index = end_index + 1
                if end_index == max_length:
                    result.append(Sentence[start_index: ])
                    # print(result[-1])
            else: 
                if Sentence[end_index] == '/' and (is_num(Sentence[end_index+1]) or is_alpbet(Sentence[end_index+1])):
                    end_index = end_index + 1
                    continue
                result.append(Sentence[start_index: end_index])
                # print(result[-1])
                start_index = end_index
                end_index = start_index + 1
        else: 
            start_index = start_index + 1
            end_index = start_index + 1
        
    return result

In [8]:
text = split(str(raw_Free_text[0]))
raw_Free_text[0], text

('No known allergies 1. Lung adenocarcinoma with pleural and BRAIN mets (TPS 60%, MET polysomy, ALK IHC high but FISH negative, EGFR exon 20 insertion (NM_005228.3) p.A763_Y764insFQEA)) to pleural, hilar and mediastinal nodes - Afatinib from 7/11/20 (dose reduced to 30mg on 27/11/20) - CTNO155X2101 May 2022 > EOT 9/9/22 - Bayer21607 22/9/22 > EOT 9/1/23 - Alimta carboplatin from 20/1/23 to 28/4/23 - WBRT June 2023 - Gemcitabine carboplatin from 6/7/23 to 11/1/24 2. Ovarian cyst',
 ['No',
  'known',
  'allergies',
  '1',
  'Lung',
  'adenocarcinoma',
  'with',
  'pleural',
  'and',
  'BRAIN',
  'mets',
  'TPS',
  '60',
  'MET',
  'polysomy',
  'ALK',
  'IHC',
  'high',
  'but',
  'FISH',
  'negative',
  'EGFR',
  'exon',
  '20',
  'insertion',
  'NM',
  '005228',
  '3',
  'p',
  'A763',
  'Y764insFQEA',
  'to',
  'pleural',
  'hilar',
  'and',
  'mediastinal',
  'nodes',
  'Afatinib',
  'from',
  '7/11/20',
  'dose',
  'reduced',
  'to',
  '30mg',
  'on',
  '27/11/20',
  'CTNO155X2101',

In [9]:
Free_text = []
# i = 0
for ft in raw_Free_text:
    Free_text.append(split(str(ft)))
    # print(i)
    # i = i+ 1
# raw_Free_text[0], Free_text[0]
    
diagnosis = []
for d in raw_diagnosis:
    diagnosis.append(split(d))

stage = []
for s in raw_stage:
    stage.append(split(str(s)))

prior_therapy = []
for pt in raw_prior_therapy:
    prior_therapy.append(split(pt))

genetic_mutation = []
for gm in raw_genetic_mutation:
    genetic_mutation.append(split(str(gm)))

additional_remark = []
for ar in raw_additional_remark:
    additional_remark.append(split(str(ar)))


In [10]:
len( Free_text)

46

## generate label collection

In [11]:
D = []
for sli in diagnosis:
    for word in sli:
        if word not in D:
            D.append(word)

S = []
for sli in stage:
    for word in sli:
        if word not in S:
            S.append(word)

P = []
for sli in prior_therapy:
    for word in sli:
        if word not in P:
            P.append(word)

G = []
for sli in genetic_mutation:
    for word in sli:
        if word not in G:
            G.append(word)

A = []
for sli in additional_remark:
    for word in sli:
        if word not in A:
            A.append(word)

#### have question here about the splite, like how do we handle with such things: 'ALK/ROS1/BRAF/MET/HER2/RET/NTRK/KRAS' in list G
#### - using number recognise to solve it, but what are the missing datas we are facing

In [12]:
diagnosis = pd.DataFrame(D, columns=['value'])
stage = pd.DataFrame(S, columns=['value'])
prior_therapy = pd.DataFrame(P, columns=['value'])
genetic_mutation = pd.DataFrame(G, columns=['value'])
additional_remark = pd.DataFrame(A, columns=['value'])

In [14]:
diagnosis.to_csv('../datasets/NER/LOCAL/diagnosis.csv', index=False)
stage.to_csv('../datasets/NER/LOCAL/stage.csv', index=False)
prior_therapy.to_csv('../datasets/NER/LOCAL/prior_therapy.csv', index=False)
genetic_mutation.to_csv('../datasets/NER/LOCAL/genetic_mutation.csv', index=False)
additional_remark.to_csv('../datasets/NER/LOCAL/additional_remark.csv', index=False)

## generate the tage based on the matching

### Like what I see in the conll DataBase, the format we need to copy
##### 'O' - the token does not need attention
##### 'B-D' - the token that presents the BEGINNING of the diagnosis
##### 'I-D' - the token that presents IN a part of the diagnosis
##### 'B-S' - the token that presents the BEGINNING of the stage
##### 'I-S' - the token that presents IN a part of the stage
##### 'B-P' - the token that presents the BEGINNING of the prior_therapy
##### 'I-P' - the token that presents IN a part of the prior_therapy
##### 'B-G' - the token that presents the BEGINNING of the genetic_mutation
##### 'I-G' - the token that presents IN a part of the genetic_mutation
##### 'B-A' - the token that presents the BEGINNING of the additional_remark
##### 'I-A' - the token that presents IN a part of the additional_remark

In [15]:
tag = []

for sentence in Free_text:
    temp_tag = []
    for word in sentence:
        now_tag = 'O'
        if word in D:
            if len(temp_tag) == 0:
                now_tag = 'B-D'
            else:
                if temp_tag[-1] == 'B-D' or temp_tag[-1] == 'I-D':
                    now_tag = 'I-D'
                else: now_tag = 'B-D'
                    
        elif word in S:
            if len(temp_tag) == 0:
                now_tag = 'B-S'
            else:
                if temp_tag[-1] == 'B-S' or temp_tag[-1] == 'I-S':
                    now_tag = 'I-S'
                else: now_tag = 'B-S'
                    
        elif word in P:
            if len(temp_tag) == 0:
                now_tag = 'B-P'
            else:
                if temp_tag[-1] == 'B-P' or temp_tag[-1] == 'I-P':
                    now_tag = 'I-P'
                else: now_tag = 'B-P'
                    
        elif word in G:
            if len(temp_tag) == 0:
                now_tag = 'B-G'
            else:
                if temp_tag[-1] == 'B-G' or temp_tag[-1] == 'I-G':
                    now_tag = 'I-G'
                else: now_tag = 'B-G'
                    
        elif word in A:
            if len(temp_tag) == 0:
                now_tag = 'B-A'
            else:
                if temp_tag[-1] == 'B-A' or temp_tag[-1] == 'I-A':
                    now_tag = 'I-A'
                else: now_tag = 'B-A'
                    
        else: now_tag = 'O'
        temp_tag.append(now_tag)
    tag.append(temp_tag)

tag[0], Free_text[0], len(tag[0]), len(Free_text[0])

(['O',
  'O',
  'O',
  'B-P',
  'B-D',
  'I-D',
  'B-S',
  'I-S',
  'I-S',
  'I-S',
  'I-S',
  'B-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'I-G',
  'B-P',
  'B-G',
  'I-G',
  'I-G',
  'B-S',
  'I-S',
  'O',
  'B-S',
  'B-P',
  'O',
  'B-P',
  'B-D',
  'O',
  'O',
  'O',
  'B-S',
  'O',
  'B-G',
  'O',
  'B-P',
  'O',
  'O',
  'O',
  'O',
  'B-P',
  'O',
  'O',
  'O',
  'B-P',
  'I-P',
  'B-D',
  'O',
  'B-S',
  'O',
  'B-P',
  'O',
  'O',
  'B-P',
  'I-P',
  'B-D',
  'O',
  'B-S',
  'O',
  'B-G',
  'O',
  'O'],
 ['No',
  'known',
  'allergies',
  '1',
  'Lung',
  'adenocarcinoma',
  'with',
  'pleural',
  'and',
  'BRAIN',
  'mets',
  'TPS',
  '60',
  'MET',
  'polysomy',
  'ALK',
  'IHC',
  'high',
  'but',
  'FISH',
  'negative',
  'EGFR',
  'exon',
  '20',
  'insertion',
  'NM',
  '005228',
  '3',
  'p',
  'A763',
  'Y764insFQEA',
  'to',
  'pleural',
  'hilar',
  'and',
  'mediastinal',
  'no

In [16]:
len(tag[21]), len(Free_text[21])

(23, 23)

In [19]:
##### 'O' - the token does not need attention
##### 'B-D' - the token that presents the BEGINNING of the diagnosis
##### 'I-D' - the token that presents IN a part of the diagnosis
##### 'B-S' - the token that presents the BEGINNING of the stage
##### 'I-S' - the token that presents IN a part of the stage
##### 'B-P' - the token that presents the BEGINNING of the prior_therapy
##### 'I-P' - the token that presents IN a part of the prior_therapy
##### 'B-G' - the token that presents the BEGINNING of the genetic_mutation
##### 'I-G' - the token that presents IN a part of the genetic_mutation
##### 'B-A' - the token that presents the BEGINNING of the additional_remark
##### 'I-A' - the token that presents IN a part of the additional_remarkwith open("../datasets/NER/LOCAL/train.txt", 'w') as file:
with open("../datasets/NER/LOCAL/labels.txt", 'w') as file:
    file.write('O')
    file.write('\n')
    file.write('B-D')
    file.write('\n')
    file.write('I-D')
    file.write('\n')
    file.write('B-S')
    file.write('\n')
    file.write('I-S')
    file.write('\n')
    file.write('B-P')
    file.write('\n')
    file.write('I-P')
    file.write('\n')
    file.write('B-G')
    file.write('\n')
    file.write('I-G')
    file.write('\n')
    file.write('B-A')
    file.write('\n')
    file.write('I-A')

In [21]:
with open("../datasets/NER/LOCAL/train.txt", 'w') as file:
    for sentence in range(len(Free_text)):
        for i in range(len(Free_text[sentence])):
            file.write(Free_text[sentence][i] + ' ' + tag[sentence][i] + '\n')
        file.write('\n')
    file.close()